In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

pwd = os.getcwd() 
train_dir = os.path.join(pwd,"temp_for_horse_human","train")
validation_dir = os.path.join(pwd,"temp_for_horse_human","validation")

train_datagen = ImageDataGenerator(rescale =1/255)
validation_datagen = ImageDataGenerator(rescale =1/255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size = (300,300), batch_size = 32, class_mode= 'binary')
validation_generator = train_datagen.flow_from_directory(validation_dir,target_size = (300,300), batch_size = 32, class_mode = 'binary')


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [7]:
from msilib.schema import Directory
import tensorflow as tf


In [8]:
pip install keras-tuner --upgrade


^C
Note: you may need to restart the kernel to use updated packages.


In [11]:
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

hp = HyperParameters()

def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(hp.Choice('num_filter_layer0',values=[16,64],default = 16), (3,3), activation = 'relu', input_shape = (300,300,3)))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    for i in range(hp.Int('num_conv_layers',1,3,step=1)):
        model.add(tf.keras.layers.Conv2D(hp.Choice(f'num_filter_layer{i}',values=[16,64],default = 16), (3,3), activation = 'relu'))
        model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hp.Int('hidden_units',128,512,step = 32), activation = 'relu'))
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr=0.001), metrics = ['acc'])
    return model

tuner = Hyperband(
    build_model,
    objective = 'val_acc',
    max_epochs = 15,
    directory = 'horse_human_parameters',
    hyperparameters = hp,
    project_name = 'my_horse_human_project'
)

tuner.search(train_generator, epochs = 10, validation_data = validation_generator)
#model.fit(train_generator, epochs = 10, validation_data = validation_generator)


Trial 30 Complete [00h 13m 30s]
val_acc: 0.89453125

Best val_acc So Far: 0.93359375
Total elapsed time: 19h 33m 57s
INFO:tensorflow:Oracle triggered exit


In [14]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)
model = tuner.hypermodel.build(best_hps)
model.summary()

{'num_filter_layer0': 64, 'num_conv_layers': 3, 'hidden_units': 448, 'num_filter_layer1': 16, 'num_filter_layer2': 16, 'tuner/epochs': 15, 'tuner/initial_epoch': 5, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0022'}
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 298, 298, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 149, 149, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 147, 147, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                       